# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [14]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_long_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/{}_per_video_trial_labels.xlsx".format(OUTPUT_PREFIX))

In [17]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in TRIAL_LABELS_DF if "unnamed" in col.lower()], errors="ignore")

In [18]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.rename(columns={col: col.strip() for col in TRIAL_LABELS_DF}, errors="ignore")

In [19]:
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX))
LFP_SPECTRAL_DF = pd.read_pickle("../2024_11_18_add_sleap/proc/{}_04_lfp_spike_sleap.pkl".format(OUTPUT_PREFIX))
# /nancy/user/riwata/projects/reward_comp_ext/results../../2024_11_18_add_sleap/proc/rce_pilot_3_long_comp_04_lfp_spike_sleap.pkl"

In [20]:
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX))

In [21]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=[col for col in LFP_SPECTRAL_DF if "unnamed" in col.lower()], errors="ignore")

In [22]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.rename(columns={col: col.strip() for col in LFP_SPECTRAL_DF}, errors="ignore")

In [23]:
# cluster_range_columns = pd.read_pickle("../2024_06_26_sleap_clustering/proc/cluster_range_columns.pkl")

In [24]:
# cluster_range_columns

In [25]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_lfp_spikes.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

In [26]:
def filter_spike_times(arr_2d, start, stop, padding=-1):
    """
    Filters a 2D array based on a start and stop condition and pads shorter arrays to match the length of the longest array.

    Parameters:
    arr_2d (numpy.ndarray): 2D array to be filtered and padded.
    start (int or float): Lower bound of the condition.
    stop (int or float): Upper bound of the condition.
    padding (int or float): Value to use for padding, defaults to -1.

    Returns:
    numpy.ndarray: A 2D array where each inner array has been filtered based on the condition and padded to match the length of the longest array.
    """

    # Filter each row in arr_2d based on the condition
    masked_data = [row[(row >= start) & (row < stop)] for row in arr_2d]

    # Determine the maximum length of arrays in the list
    max_length = max(len(arr) for arr in masked_data)

    # Pad each array in masked_data with the padding value so they all have the same length (max_length)
    padded_arrays = [np.concatenate([x, np.full([max_length - len(x)], padding)]) for x in masked_data]

    # Convert the list of arrays to a 2D numpy array
    padded_arrays = np.array(padded_arrays)

    return padded_arrays

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [27]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [28]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'notes', 'session_path',
       'recording', 'current_subject', 'subject', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'vHPC_lfp_trace',
       'BLA_lfp_trace', 'LH_lfp_trace', 'MD_lfp_trace', 'mPFC_lfp_trace',
       'recording_name', 'spike_clusters', 'spike_times', 'neuron_average_fr',
       'neuron_average_timestamps', 'start_frame', 'stop_frame',
       'tracked_subject', 'in_video_subjects', 'box_number', 'sleap_name',
       'body_parts', 'box_bottom_right', 'reward_port', 'box_top_left',
       'box_top_right', 'box_bottom_left', 'agent', 'subject_locations',
       'frame_index', 'agent_locations', 'subject_thorax_velocity',
       'agent_thorax_velocity', 'subject_thorax_to_reward_port',
       'agent_thorax_

In [29]:
TRIAL_LABELS_DF.columns

Index(['session_dir', 'tone_start_frame', 'reward_start', 'reward_dispensed',
       'tone_stop_frame', 'condition', 'competition_closeness',
       'get_reward_frame', 'out_reward_frame', 'notes', 'experiment',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'all_subjects', 'first_timestamp', 'last_timestamp',
       'tone_start_timestamp', 'tone_stop_timestamp',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps'],
      dtype='object')

## Merging the labels and LFP spectral

In [30]:
TRIAL_LABELS_DF.head()

,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,...,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240317_151922_long_comp_subj_3-1_and_3-3,1,81,101,88,NaN,NaN,NaN,NaN,NaN,...,[ 1 88],NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,0,87540,[ 0 87540],NaN
1,20240317_151922_long_comp_subj_3-1_and_3-3,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,NaN,...,[1300 1315] [1371 1380] [1420 1513],NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,1287955,1487958,[1301956 1317753] [1373156 1382756] [1422554 1...,NaN
2,20240317_151922_long_comp_subj_3-1_and_3-3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,NaN,...,[3277 3298] [3310 3356] [3357 3368] [3377 3378...,NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,3287980,3487980,[3281780 3302780] [3315978 3361781] [3362381 3...,NaN
3,20240317_151922_long_comp_subj_3-1_and_3-3,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,"3.3 walk towards the reward port, and then wal...",...,[4383 4502] [4527 4851],NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,4487995,4687995,[4390791 4509995] [4535398 4859200],NaN
4,20240317_151922_long_comp_subj_3-1_and_3-3,5478,5558,5578,5678,3.1,Subj 1 blocking Subj 2,NaN,NaN,3.1 blocked 3.3 from approaching at frame 5574,...,[5588 5778],NaN,20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,"['3.1', '3.3']",2415571,48236575,5488007,5688010,[5597806 5788609],NaN


In [31]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,box_top_right,box_bottom_left,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-1_t6b6_merged,3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03..."
1,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-3_t5b5_merged,3.3,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.1,"[[[51.049592291310574, 19.27532173433335], [54...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[58.961398589179765, 35.28935156776581], [57...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965..."
2,rce_pilot_3,20240317_172017_long_comp_subj_4-2_and_4-3,"[[1, 102], [1300, 1501], [3297, 3497], [4494, ...","[[1, 102], [115, 153], [191, 245], [261, 263],...",[],20240317_172017_long_comp_subj_4-2_and_4-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240317_172017_long_comp_subj_4-2_t6b6_merged,4.2,...,"[73.77902458765409, 9.138890215293802]","[45.35645871188365, 33.78665543178387]",4.3,"[[[62.54046053502408, 34.646917855927946], [59...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[59.070922225644495, 9.032051361034663], [61...","[1.537, 1.132, 0.8584, 0.811, 0.987, 1.272, 1....","[15.766, 15.34, 15.016, 14.79, 14.66, 14.59, 1...","[5.25, 5.43, 5.523, 5.566, 5.547, 5.46, 5.258,...","[26.3, 26.12, 26.12, 26.12, 26.14, 26.17, 26.2..."
3,rce_pilot_3,20240317_172017_long_comp_subj_4-2_and_4-3,"[[1, 102], [1300, 1501], [3297, 3497], [4494, ...","[[1, 102], [115, 153], [191, 245], [261, 263],...",[],20240317_172017_long_comp_subj_4-2_and_4-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240317_172017_long_comp_subj_4-3_t5b5_merged,4.3,...,"[73.77902458765409, 9.138890215293802]","[45.35645871188365, 33.78665543178387]",4.2,"[[[59.070922225644495, 9.032051361034663], [61...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[62.54046053502408, 34.646917855927946], [59...","[15.766, 15.34, 15.016, 14.79, 14.66, 14.59, 1...","[1.537, 1.132, 0.8584, 0.811, 0.987, 1.272, 1....","[26.3, 26.12, 26.12, 26.12, 26.14, 26.17, 26.2...","[5.25, 5.43, 5.523, 5.566, 5.547, 5.46, 5.258,..."
4,rce_pilot_3,20240318_143819_long_comp_subj_3-3_and_3-4,"[[1, 125], [1334, 1533], [3330, 3529], [4528, ...","[[1, 125], [125, 181], [369, 401], [403, 446],...",[],20240318_143819_long_comp_subj_3-3_and_3-4.1,NaN,/scratch/back_up/reward_competition_extention/...,20240318_143819_long_comp_subj_3-3_t6b6_merged,3.3,...,"[74.70618240859957, 10.259858561906261]","[45.51706003228968, 34.29512637682901]",3.4,"[[[76.32303058013719, 9.877634998966064], [79....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[41.7841383440293, 12.629758340372002], [42....","[5.914, 5.984, 6.203, 6.54, 6.96, 7.

# Dropping redundant columns

In [32]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [33]:
dropping_columns

['notes',
 'last_timestamp',
 'video_name',
 'box_1_port_entry_timestamps',
 'session_dir',
 'box_2_port_entry_timestamps',
 'all_subjects',
 'box_2_port_entry_frames',
 'first_timestamp',
 'box_1_port_entry_frames']

In [34]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=[col for col in dropping_columns if col != "video_name"], errors="ignore")

## Merging the LFP and trial dataframe

In [35]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [36]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240317_151922_long_comp_subj_3-1_and_3-3.1',
       '20240317_172017_long_comp_subj_4-2_and_4-3.1',
       '20240318_143819_long_comp_subj_3-3_and_3-4.1',
       '20240318_170933_long_comp_subj_4-3_and_4-4.1',
       '20240319_134914_long_comp_subj_3-1_and_3-4.1',
       '20240319_160457_long_comp_subj_4-2_and_4-4.1',
       '20240320_114629_long_comp_subj_5-3_and_5-4.1',
       '20240321_114851_long_comp_subj_5-2_and_5-3.1'], dtype=object)

In [37]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)


In [38]:
LFP_SPECTRAL_DF["current_subject"].head()

0    3.1
1    3.3
2    4.2
3    4.3
4    3.3
Name: current_subject, dtype: object

In [39]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["video_name"])

In [40]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition"])

In [41]:
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [42]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240317_151922_long_comp_subj_3-1_and_3-3.1',
       '20240317_172017_long_comp_subj_4-2_and_4-3.1',
       '20240318_143819_long_comp_subj_3-3_and_3-4.1',
       '20240318_170933_long_comp_subj_4-3_and_4-4.1',
       '20240319_134914_long_comp_subj_3-1_and_3-4.1',
       '20240319_160457_long_comp_subj_4-2_and_4-4.1',
       '20240320_114629_long_comp_subj_5-3_and_5-4.1',
       '20240321_114851_long_comp_subj_5-2_and_5-3.1'], dtype=object)

In [43]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["video_name"], how="inner", suffixes=('', '_remove'))

In [44]:
TRIALS_AND_SPECTRAL_DF[[col for col in TRIALS_AND_SPECTRAL_DF if "_remove" in col]].head()

""
0
1
2
3
4


In [45]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "remove" in col], errors="ignore")

In [46]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "Unnamed" in col])

In [47]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,box_top_right,box_bottom_left,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03..."
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.1,"[[[51.049592291310574, 19.27532173433335], [54...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[58.961398589179765, 35.28935156776581], [57...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965..."
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03..."
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.1,"[[[51.049592291310574, 19.27532173433335], [54...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[58.961398589179765, 35.28935156776581], [57...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965..."
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03..."


In [48]:
TRIALS_AND_SPECTRAL_DF.tail()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,box_top_right,box_bottom_left,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
465,49514,49594,49614,49713,5.3,Subj 2 then Subj 1,49592.0,49703.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[74.19640018846134, 8.857052224900107]","[46.06164691686389, 33.503473726463156]",5.2,"[[[75.1770338506859, 29.115218464800385], [71....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[62.95310346293313, 34.789092579377574], [60...","[15.84, 14.85, 13.875, 12.92, 11.984, 11.08, 1...","[55.9, 52.1, 48.4, 44.8, 41.3, 37.94, 34.66, 3...","[15.25, 14.89, 14.95, 15.42, 15.336, 14.58, 14...","[5.36, 5.805, 6.09, 5.97, 5.918, 5.715, 5.426,..."
466,50511,50591,50611,50711,5.3,Subj 2 blocking Subj 1,50498.0,50676.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[74.19640018846134, 8.857052224900107]","[46.06164691686389, 33.503473726463156]",5.2,"[[[75.1770338506859, 29.115218464800385], [71....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[62.95310346293313, 34.789092579377574], [60...","[15.84, 14.85, 13.875, 12.92, 11.984, 11.08, 1...","[55.9, 52.1, 48.4, 44.8, 41.3, 37.94, 34.66, 3...","[15.25, 14.89, 14.95, 15.42, 15.336, 14.58, 14...","[5.36, 5.805, 6.09, 5.97, 5.918, 5.715, 5.426,..."
467,51909,51989,52009,52108,5.3,Subj 2 then Subj 1,51972.0,52121.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[74.19640018846134, 8.857052224900107]","[46.06164691686389, 33.503473726463156]",5.2,"[[[75.1770338506859, 29.115218464800385], [71....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[62.95310346293313, 34.789092579377574], [60...","[15.84, 14.85, 13.875, 12.92, 11.984, 11.08, 1...","[55.9, 52.1, 48.4, 44.8, 41.3, 37.94, 34.66, 3...","[15.25, 14.89, 14.95, 15.42, 15.336, 14.58, 14...","[5.36, 5.805, 6.09, 5.97, 5.918, 5.715, 5.426,..."
468,54104,54184,54204,54304,5.3,Subj 2 then Subj 1,54157.0,54286.0,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[74.19640018846134, 8.857052224900107]","[46.06164691686389, 33.503473726463156]",5.2,"[[[75.1770338506859, 29.115218464800385], [71....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[62.95310346293313, 34.789092579377574], [60...","[15.84, 14.85, 13.875, 12.92, 11.984, 11.08, 1...","[55.9, 52.1, 48.4, 44.8, 41.3, 37.94, 34.66, 3...","[15.25, 14.89, 14.95, 15.42, 15.336, 14.58, 14...","[5.36, 5.805, 6.09, 5.97, 5.918, 5.715, 5.426,..."
469,55203,55283,55303,55402,tie,Close Call,NaN,NaN,long_comp,20240321_114851_long_comp_subj_5-2_and_5-3.1,...,"[74.19640018846134, 8.857052224900107]","[46.06164691686389, 33.503473726463156]",5.2,"[[[75.1770338506859, 29.115218464800385], [71....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[62.95310346293313, 34.789092579377574], [60...","[15.84, 14.85, 13.875, 12.92, 11.984, 11.08, 1...","[55.9, 52.1, 48.4, 44.8, 41.3, 37.94, 34.66, 3...","[15.25, 14.89, 14.95, 15.42, 15.336, 14.58, 14...","[5.36, 5.805, 6.09, 5.97, 5.918, 5.715, 5.426,..."


In [49]:
timestamps_col = [col for col in TRIALS_AND_SPECTRAL_DF if "timestamp" in col and "timestamps" not in col and "cluster" not in col]

In [50]:
for col in timestamps_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [51]:
TRIALS_AND_SPECTRAL_DF[timestamps_col].head()

,tone_start_timestamp,tone_stop_timestamp,first_timestamp,last_timestamp
0,1287955,1487958,2415571,48236575
1,1287955,1487958,2415571,48236575
2,3287980,3487980,2415571,48236575
3,3287980,3487980,2415571,48236575
4,4487995,4687995,2415571,48236575


In [52]:
frame_col = [col for col in TRIALS_AND_SPECTRAL_DF if "frame" in col and "tone" in col and "frames" not in col]

In [53]:
for col in frame_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [54]:
TRIALS_AND_SPECTRAL_DF[frame_col].head()

,tone_start_frame,tone_stop_frame
0,1286,1486
1,1286,1486
2,3282,3482
3,3282,3482
4,4480,4679


## Adding the SLEAP data

In [55]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,box_top_right,box_bottom_left,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03..."
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.1,"[[[51.049592291310574, 19.27532173433335], [54...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[58.961398589179765, 35.28935156776581], [57...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965..."
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03..."
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.1,"[[[51.049592291310574, 19.27532173433335], [54...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[58.961398589179765, 35.28935156776581], [57...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965..."
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[73.57122102264293, 11.423230211638582]","[45.03093731344292, 35.761309141917415]",3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03..."


In [56]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1287955
1    1287955
2    3287980
3    3287980
4    4487995
Name: tone_start_timestamp, dtype: int64

In [57]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].astype(int)

In [58]:
TRIALS_AND_SPECTRAL_DF["current_subject"] = TRIALS_AND_SPECTRAL_DF["current_subject"].apply(lambda x: str(x).strip())

# Calculating the relevant timestamps

In [59]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 10 * EPHYS_SAMPLE_RATE

In [60]:
TRIALS_AND_SPECTRAL_DF["post_trial_end_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"] + 10 * EPHYS_SAMPLE_RATE

In [61]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1287955
1    1287955
2    3287980
3    3287980
4    4487995
Name: tone_start_timestamp, dtype: int64

In [62]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1287955
1    1287955
2    3287980
3    3287980
4    4487995
Name: tone_start_timestamp, dtype: int64

In [63]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    1487958
1    1487958
2    3487980
3    3487980
4    4687995
Name: tone_stop_timestamp, dtype: int64

In [64]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,agent,subject_locations,frame_index,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port,baseline_start_timestamp,post_trial_end_timestamp
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...",1087955,1687958
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,3.1,"[[[51.049592291310574, 19.27532173433335], [54...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[58.961398589179765, 35.28935156776581], [57...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...",1087955,1687958
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...",3087980,3687980
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,3.1,"[[[51.049592291310574, 19.27532173433335], [54...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[58.961398589179765, 35.28935156776581], [57...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...",3087980,3687980
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,3.3,"[[[58.961398589179765, 35.28935156776581], [57...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[[51.049592291310574, 19.27532173433335], [54...","[3.035, 2.855, 2.68, 2.512, 2.348, 2.19, 2.037...","[12.37, 11.99, 11.65, 11.336, 11.06, 10.81, 10...","[7.453, 7.25, 7.273, 7.062, 6.88, 6.617, 5.965...","[19.94, 20.16, 20.45, 20.8, 21.0, 20.98, 21.03...",4287995,4887995


# Filtering out the SLEAP posed for during trials

In [65]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [66]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head().apply(lambda x: x.shape)

0    (45736,)
1    (45736,)
2    (45736,)
3    (45736,)
4    (45736,)
Name: video_frame, dtype: object

In [67]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if ("thorax" in col or "nose" in col or "standard_embedding" in col or "frame_index" in col or "locations" in col or "tailbase" in col or "competitiveness" in col) and "timestamp" not in col and "dict" not in col]


In [68]:
sleap_columns

['subject_locations',
 'frame_index',
 'agent_locations',
 'subject_thorax_velocity',
 'agent_thorax_velocity',
 'subject_thorax_to_reward_port',
 'agent_thorax_to_reward_port']

In [69]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"][col].apply(lambda x: x.shape if isinstance(x, np.ndarray) else np.nan).head())

subject_locations
0    (45736, 6, 2)
1    (45736, 6, 2)
2    (45736, 6, 2)
3    (45736, 6, 2)
4    (45736, 6, 2)
Name: subject_locations, dtype: object
frame_index
0    (45736,)
1    (45736,)
2    (45736,)
3    (45736,)
4    (45736,)
Name: frame_index, dtype: object
agent_locations
0    (45736, 6, 2)
1    (45736, 6, 2)
2    (45736, 6, 2)
3    (45736, 6, 2)
4    (45736, 6, 2)
Name: agent_locations, dtype: object
subject_thorax_velocity
0    (45736,)
1    (45736,)
2    (45736,)
3    (45736,)
4    (45736,)
Name: subject_thorax_velocity, dtype: object
agent_thorax_velocity
0    (45736,)
1    (45736,)
2    (45736,)
3    (45736,)
4    (45736,)
Name: agent_thorax_velocity, dtype: object
subject_thorax_to_reward_port
0    (45736,)
1    (45736,)
2    (45736,)
3    (45736,)
4    (45736,)
Name: subject_thorax_to_reward_port, dtype: object
agent_thorax_to_reward_port
0    (45736,)
1    (45736,)
2    (45736,)
3    (45736,)
4    (45736,)
Name: agent_thorax_to_reward_port, dtype: object


In [70]:
TRIALS_AND_SPECTRAL_DF.columns

Index(['tone_start_frame', 'reward_start', 'reward_dispensed',
       'tone_stop_frame', 'condition', 'competition_closeness',
       'get_reward_frame', 'out_reward_frame', 'experiment', 'video_name',
       'tone_start_timestamp', 'tone_stop_timestamp', 'cohort', 'session_dir',
       'tone_frames', 'box_1_port_entry_frames', 'box_2_port_entry_frames',
       'notes', 'session_path', 'recording', 'current_subject', 'subject',
       'all_subjects', 'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'vHPC_lfp_trace',
       'BLA_lfp_trace', 'LH_lfp_trace', 'MD_lfp_trace', 'mPFC_lfp_trace',
       'recording_name', 'spike_clusters', 'spike_times', 'neuron_average_fr',
       'neuron_average_timestamps', 'start_frame', 'stop_frame',
       'tracked_subject', 'in_video_subjects', 'box_number', 'sleap_name',
       'body_parts', 'box_bottom_right', 'reward_port', 'box_top

In [71]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if isinstance(x["video_timestamps"], np.ndarray) else np.nan, axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0] if isinstance(x["video_timestamps"], np.ndarray) else np.nan, axis=1)

trial_agent_locations
trial_agent_thorax_to_reward_port
trial_agent_thorax_velocity
trial_frame_index
trial_subject_locations
trial_subject_thorax_to_reward_port
trial_subject_thorax_velocity


In [72]:
for col in sorted(sleap_columns):
    updated_item_col = "baseline_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "baseline_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if isinstance(x["video_timestamps"], np.ndarray) else np.nan, axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0] if isinstance(x["video_timestamps"], np.ndarray) else np.nan, axis=1)

baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baseline_frame_index
baseline_subject_locations
baseline_subject_thorax_to_reward_port
baseline_subject_thorax_velocity


In [73]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"].head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,trial_subject_thorax_velocity,trial_video_timestamps,baseline_agent_locations,baseline_agent_thorax_to_reward_port,baseline_agent_thorax_velocity,baseline_frame_index,baseline_subject_locations,baseline_subject_thorax_to_reward_port,baseline_subject_thorax_velocity,baseline_video_timestamps
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[2.088, 2.21, 2.37, 2.525, 2.686, 2.781, 2.83,...","[1288857, 1288857, 1290243, 1291629, 1291629, ...","[[[47.32329934790797, 13.40804063234078], [46....","[27.48, 27.6, 27.6, 27.6, 27.61, 27.61, 27.61,...","[3.97, 3.36, 2.844, 2.33, 1.876, 1.488, 1.177,...","[1088.0, 1089.0, 1090.0, 1091.0, 1092.0, 1093....","[[[61.61018887260965, 36.23981343180496], [58....","[5.04, 5.047, 5.043, 5.03, 5.04, 5.04, 5.03, 5...","[0.592, 0.535, 0.4746, 0.4119, 0.3462, 0.2942,...","[1089292, 1089292, 1090678, 1092063, 1093449, ..."
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.8057, 1.038, 1.263, 1.474, 1.6455, 1.753, 1...","[1288857, 1288857, 1290243, 1291629, 1291629, ...","[[[61.61018887260965, 36.23981343180496], [58....","[5.04, 5.047, 5.043, 5.03, 5.04, 5.04, 5.03, 5...","[0.592, 0.535, 0.4746, 0.4119, 0.3462, 0.2942,...","[1088.0, 1089.0, 1090.0, 1091.0, 1092.0, 1093....","[[[47.32329934790797, 13.40804063234078], [46....","[27.48, 27.6, 27.6, 27.6, 27.61, 27.61, 27.61,...","[3.97, 3.36, 2.844, 2.33, 1.876, 1.488, 1.177,...","[1089292, 1089292, 1090678, 1092063, 1093449, ..."
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[1.07, 1.044, 0.979, 0.887, 0.7646, 0.613, 0.4...","[3288667, 3288667, 3290053, 3291438, 3292824, ...","[[[64.57511078487119, 13.578025427299137], [67...","[24.42, 24.67, 24.72, 24.6, 24.5, 24.58, 24.62...","[7.008, 7.08, 7.145, 7.207, 7.258, 7.23, 7.113...","[3084.0, 3085.0, 3086.0, 3087.0, 3088.0, 3089....","[[[49.56347454661353, 39.209700156326015], [46...","[10.914, 10.92, 10.73, 10.5, 10.69, 10.9, 10.8...","[3.572, 3.482, 3.357, 3.195, 3.0, 2.781, 2.535...","[3089101, 3089101, 3090487, 3091873, 3093259, ..."
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[8.75, 8.47, 8.16, 7.836, 7.48, 7.156, 6.793, ...","[3288667, 3288667, 3290053, 3291438, 3292824, ...","[[[49.56347454661353, 39.209700156326015], [46...","[10.914, 10.92, 10.73, 10.5, 10.69, 10.9, 10.8...","[3.572, 3.482, 3.357, 3.195, 3.0, 2.781, 2.535...","[3084.0, 3085.0, 3086.0, 3087.0, 3088.0, 3089....","[[[64.57511078487119, 13.578025427299137], [67...","[24.42, 24.67, 24.72, 24.6, 24.5, 24.58, 24.62...","[7.008, 7.08, 7.145, 7.207, 7.258, 7.23, 7.113...","[3089101, 3089101, 3090487, 3091873, 3093259, ..."
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[0.211, 0.2566, 0.276, 0.3174, 0.3132, 0.2764,...","[4488830, 4488830, 4490216, 4491601, 4492351, ...","[[[62.851787097898, 36.10868185024038], [61.39...","[6.305, 6.457, 6.457, 6.453, 6.227, 5.785, 5.3...","[1.689, 1.897, 2.055, 2.193, 2.31, 2.404, 2.47...","[4282.0, 4283.0, 4284.0, 4285.0, 4286.0, 4287....","[[[76.40203405981883, 16.48636114635069], [71....","[28.27, 26.19, 25.61, 25.22, 24.17, 22.88, 21....","[13.46, 13.79, 14.055, 14.27, 14.47, 14.62, 14...","[4289264, 4290650, 4290650, 4292036, 4293422, ..."


In [74]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["trial_frame_index"].iloc[0].shape

(200,)

In [75]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["video_timestamps"], errors="ignore")

In [76]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
recording_name
spike_clusters
spike_times
neuron_average_fr
neuron_average_timestamps
start_frame
stop_frame
tracked_subject
in_video_subjects
box_number
sleap_name
body_parts
box_bottom_right
reward_port
box_top_left
box_top_right
box_bottom_left
agent
baseline_start_timestamp
post_trial_end_timestamp
video_frame
trial_agent_locations
trial_agent_thorax_to_reward_port
trial_agent_thorax_velocity
trial_frame_index
trial_subject_locations
trial_subject_thorax_to_reward_po

# Filtering out spikes

In [77]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"]).astype(int), axis=1)


In [78]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [79]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    1287955
1    1287955
2    3287980
3    3287980
4    4487995
Name: tone_start_timestamp, dtype: int64

In [80]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

0    1087955
1    1087955
2    3087980
3    3087980
4    4287995
Name: baseline_start_timestamp, dtype: int64

In [81]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

array([[1103189, 1109287, 1110845, ...,      -1,      -1,      -1],
       [1095217, 1112469, 1171563, ...,      -1,      -1,      -1],
       [1206532, 1217592, 1217739, ...,      -1,      -1,      -1],
       ...,
       [1286593,      -1,      -1, ...,      -1,      -1,      -1],
       [1088149, 1089336, 1090000, ..., 1286711, 1287104, 1287638],
       [1088616, 1091484, 1093964, ...,      -1,      -1,      -1]])

In [82]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [83]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [84]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [85]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

(7, 100)

In [86]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape

(100,)

- Filtering coherence

In [87]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [88]:
coherence_columns

[]

In [89]:
try:
    for col in sorted(coherence_columns):
        updated_item_col = "trial_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "trial_coherence_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [90]:
try:
    for col in sorted(coherence_columns):
        updated_item_col = "baseline_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "baseline_coherence_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [91]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [92]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [93]:
granger_columns

[]

In [94]:
try:
    for col in sorted(granger_columns):
        updated_item_col = "trial_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "trial_granger_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [95]:
try:
    for col in sorted(granger_columns):
        updated_item_col = "baseline_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "baseline_granger_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [96]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

- Filtering power

In [97]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,video_name,...,baseline_subject_locations,baseline_subject_thorax_to_reward_port,baseline_subject_thorax_velocity,baseline_video_timestamps,baseline_spike_times,trial_spike_times,baseline_neuron_average_fr,baseline_neuron_average_timestamp,trial_neuron_average_fr,trial_neuron_average_timestamp
0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[61.61018887260965, 36.23981343180496], [58....","[5.04, 5.047, 5.043, 5.03, 5.04, 5.04, 5.03, 5...","[0.592, 0.535, 0.4746, 0.4119, 0.3462, 0.2942,...","[1089292, 1089292, 1090678, 1092063, 1093449, ...","[[1103189, 1109287, 1110845, 1114071, 1115581,...","[[1289740, 1294804, 1297689, 1302014, 1302321,...","[[0.0, 0.0, 0.0, 1.0, 0.0, 2.0, 1.0], [0.0, 0....","[1088000, 1090000, 1092000, 1094000, 1096000, ...","[[1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 3.0, 0.0,...","[1288000, 1290000, 1292000, 1294000, 1296000, ..."
1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[47.32329934790797, 13.40804063234078], [46....","[27.48, 27.6, 27.6, 27.6, 27.61, 27.61, 27.61,...","[3.97, 3.36, 2.844, 2.33, 1.876, 1.488, 1.177,...","[1089292, 1089292, 1090678, 1092063, 1093449, ...","[[1114212, 1167836, -1, -1, -1, -1, -1, -1, -1...","[[1348287, 1469314, -1, -1, -1, -1, -1, -1, -1...","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1088000, 1090000, 1092000, 1094000, 1096000, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1288000, 1290000, 1292000, 1294000, 1296000, ..."
2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[49.56347454661353, 39.209700156326015], [46...","[10.914, 10.92, 10.73, 10.5, 10.69, 10.9, 10.8...","[3.572, 3.482, 3.357, 3.195, 3.0, 2.781, 2.535...","[3089101, 3089101, 3090487, 3091873, 3093259, ...","[[3091969, 3097498, 3099161, 3101809, 3102274,...","[[3298338, 3300915, 3308044, 3312927, 3322024,...","[[0.0, 0.0, 0.0, 2.0, 0.0, 3.0, 1.0], [1.0, 0....","[3088000, 3090000, 3092000, 3094000, 3096000, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0,...","[3288000, 3290000, 3292000, 3294000, 3296000, ..."
3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[64.57511078487119, 13.578025427299137], [67...","[24.42, 24.67, 24.72, 24.6, 24.5, 24.58, 24.62...","[7.008, 7.08, 7.145, 7.207, 7.258, 7.23, 7.113...","[3089101, 3089101, 3090487, 3091873, 3093259, ...","[[3096843, -1, -1, -1, -1, -1, -1, -1, -1, -1,...","[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0,...","[3088000, 3090000, 3092000, 3094000, 3096000, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3288000, 3290000, 3292000, 3294000, 3296000, ..."
4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1,...,"[[[76.40203405981883, 16.48636114635069], [71....","[28.27, 26.19, 25.61, 25.22, 24.17, 22.88, 21....","[13.46, 13.79, 14.055, 14.27, 14.47, 14.62, 14...","[4289264, 4290650, 4290650, 4292036, 4293422, ...","[[4310086, 4312107, 4323688, 4331381, 4335574,...","[[4494910, 4500834, 4526144, 4533592, 4534776,...","[[0.0, 0.0, 1.0, 0.0, 0.0, 3.0, 1.0], [0.0, 0....","[4288000, 4290000, 4292000, 4294000, 4296000, ...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[4488000, 4490000, 4492000, 4494000, 4496000, ..."


In [98]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [99]:
power_columns

[]

In [100]:
try:
    for col in sorted(power_columns):
        updated_item_col = "trial_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "trial_power_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [101]:
try:
    for col in sorted(power_columns):
        updated_item_col = "baseline_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "baseline_power_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)
except:
    pass

In [102]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

# Filtering out phase

In [103]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [104]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [105]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [106]:
phase_columns = phase_columns + band_columns +lfp_columns

In [107]:
phase_columns

['vHPC_lfp_trace',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace']

In [108]:
try:
    for col in sorted(phase_columns):
        updated_item_col = "trial_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "trial_lfp_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)
except:
    pass

trial_BLA_lfp_trace
trial_LH_lfp_trace
trial_MD_lfp_trace
trial_mPFC_lfp_trace
trial_vHPC_lfp_trace


In [109]:
try:
    for col in sorted(phase_columns):
        updated_item_col = "baseline_{}".format(col)
        print(updated_item_col)
        updated_timestamp_col = "baseline_lfp_timestamps"
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

    TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["tone_start_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)
except:
    pass

baseline_BLA_lfp_trace
baseline_LH_lfp_trace
baseline_MD_lfp_trace
baseline_mPFC_lfp_trace
baseline_vHPC_lfp_trace


In [110]:
TRIALS_AND_SPECTRAL_DF["trial_lfp_timestamps"]

0      [1287960, 1287980, 1288000, 1288020, 1288040, ...
1      [1287960, 1287980, 1288000, 1288020, 1288040, ...
2      [3287980, 3288000, 3288020, 3288040, 3288060, ...
3      [3287980, 3288000, 3288020, 3288040, 3288060, ...
4      [4488000, 4488020, 4488040, 4488060, 4488080, ...
                             ...                        
465    [49606260, 49606280, 49606300, 49606320, 49606...
466    [50606280, 50606300, 50606320, 50606340, 50606...
467    [52006300, 52006320, 52006340, 52006360, 52006...
468    [54206320, 54206340, 54206360, 54206380, 54206...
469    [55306320, 55306340, 55306360, 55306380, 55306...
Name: trial_lfp_timestamps, Length: 470, dtype: object

In [111]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [112]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition
competition_closeness
get_reward_frame
out_reward_frame
experiment
video_name
tone_start_timestamp
tone_stop_timestamp
cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
recording_name
start_frame
stop_frame
tracked_subject
in_video_subjects
box_number
sleap_name
body_parts
box_bottom_right
reward_port
box_top_left
box_top_right
box_bottom_left
agent
baseline_start_timestamp
post_trial_end_timestamp
video_frame
trial_agent_locations
trial_agent_thorax_to_reward_port
trial_agent_thorax_velocity
trial_frame_index
trial_subject_locations
trial_subject_thorax_to_reward_port
trial_subject_thorax_velocity
trial_video_timestamps
baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baselin

In [113]:
# TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["condition"] != "rewarded"]["trial_kmeans_cluster"].iloc[0].shape

In [114]:
TRIALS_AND_SPECTRAL_DF.shape

(470, 79)

In [115]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

# Filtering for rows that are in the video

In [116]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_frame_index"][0] <= x["stop_frame"], axis=1)

In [117]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [118]:
TRIALS_AND_SPECTRAL_DF

,index,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,experiment,...,trial_mPFC_lfp_trace,trial_vHPC_lfp_trace,trial_lfp_timestamps,baseline_BLA_lfp_trace,baseline_LH_lfp_trace,baseline_MD_lfp_trace,baseline_mPFC_lfp_trace,baseline_vHPC_lfp_trace,baseline_lfp_timestamps,in_video
0,0,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,...,"[0.0, 113.0, 212.0, 294.0, 401.0, 381.0, 280.0...","[283.0, 265.0, 148.0, 128.0, 164.0, 175.0, 367...","[1287960, 1287980, 1288000, 1288020, 1288040, ...","[206.0, 52.0, 59.0, 164.0, 167.0, 100.0, 120.0...","[445.0, 160.0, 16.0, 121.0, 209.0, 136.0, 48.0...","[291.0, 5.0, -196.0, -145.0, -53.0, -51.0, -10...","[-160.0, -369.0, -411.0, -258.0, -195.0, -279....","[201.0, 261.0, 381.0, 399.0, 203.0, 30.0, 54.0...","[1087960, 1087980, 1088000, 1088020, 1088040, ...",True
1,1,1286,1366,1386,1486,3.3,Subj 2 then Subj 1,NaN,NaN,long_comp,...,"[-682.0, -861.0, -778.0, -689.0, -768.0, -966....","[-19.0, -189.0, -315.0, -239.0, -116.0, -170.0...","[1287960, 1287980, 1288000, 1288020, 1288040, ...","[222.0, 195.0, 131.0, 107.0, 149.0, 209.0, 264...","[164.0, 151.0, 69.0, 89.0, 207.0, 230.0, 153.0...","[131.0, 127.0, 92.0, 81.0, 109.0, 126.0, 134.0...","[354.0, 490.0, 414.0, 365.0, 472.0, 543.0, 532...","[417.0, 297.0, 185.0, 181.0, 307.0, 452.0, 511...","[1087960, 1087980, 1088000, 1088020, 1088040, ...",True
2,2,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,...,"[-75.0, 272.0, 245.0, -33.0, -202.0, -69.0, 14...","[1169.0, 1396.0, 1279.0, 980.0, 825.0, 946.0, ...","[3287980, 3288000, 3288020, 3288040, 3288060, ...","[-299.0, -234.0, -138.0, -207.0, -351.0, -364....","[-94.0, -23.0, 93.0, 113.0, 93.0, 144.0, 209.0...","[163.0, 239.0, 384.0, 409.0, 350.0, 378.0, 451...","[-79.0, 191.0, 361.0, 348.0, 271.0, 260.0, 334...","[-1582.0, -1426.0, -1172.0, -969.0, -756.0, -4...","[3087980, 3088000, 3088020, 3088040, 3088060, ...",True
3,3,3282,3362,3382,3482,3.1,Subj 1 Only,NaN,NaN,long_comp,...,"[465.0, 614.0, 819.0, 904.0, 821.0, 776.0, 875...","[-128.0, -134.0, -27.0, 74.0, 77.0, 46.0, 64.0...","[3287980, 3288000, 3288020, 3288040, 3288060, ...","[-33.0, 45.0, 183.0, 327.0, 406.0, 515.0, 693....","[-160.0, -52.0, 76.0, 231.0, 322.0, 406.0, 551...","[-284.0, -184.0, -131.0, 74.0, 284.0, 415.0, 5...","[-444.0, -459.0, -156.0, 217.0, 259.0, 277.0, ...","[-521.0, -346.0, -191.0, -35.0, 103.0, 266.0, ...","[3087980, 3088000, 3088020, 3088040, 3088060, ...",True
4,4,4480,4560,4580,4679,3.1,Subj 1 Only,NaN,NaN,long_comp,...,"[77.0, 75.0, 114.0, 208.0, 139.0, -58.0, -94.0...","[-215.0, -107.0, 22.0, 202.0, 393.0, 373.0, 17...","[4488000, 4488020, 4488040, 4488060, 4488080, ...","[156.0, -122.0, -181.0, 89.0, 198.0, 57.0, 67....","[408.0, 96.0, 112.0, 478.0, 628.0, 522.0, 481....","[357.0, 106.0, 116.0, 478.0, 719.0, 691.0, 609...","[429.0, 341.0, 503.0, 794.0, 907.0, 799.0, 668...","[81.0, -78.0, -108.0, 54.0, 0.0, -192.0, -237....","[4288000, 4288020, 4288040, 4288060, 4288080, ...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,465,49514,49594,49614,49713,5.3,Subj 2 then Subj 1,49592.0,49703.0,long_comp,...,"[362.0, 375.0, 489.0, 656.0, 771.0, 784.0, 776...","[-2887.0, -2540.0, -2086.0, -1840.0, -1812.0, ...","[49606260, 49606280, 49606300, 49606320, 49606...","[-506.0, -617.0, -848.0, -898.0, -624.0, -364....","[-165.0, 26.0, -43.0, -74.0, 185.0, 469.0, 559...","[-456.0, -509.0, -684.0, -761.0, -522.0, -277....","[-377.0, -261.0, -489.0, -531.0, -185.0, -26.0...","[2047.0, 2170.0, 2196.0, 2199.0, 2219.0, 2096....","[49406260, 49406280, 49406300, 49406320, 49406...",True
466,466,50511,50591,50611,50711,5.3,Subj 2 blocking Subj 1,50498.0,50676.0,long_comp,...,"[-492.0, -422.0, -288.0, -260.0, -339.0, -367....","[-3586.0, -3261.0, -2919.0, -2787.0, -2884.0, ...","[50606280, 50606300, 50606320, 50606340, 50606...","[134.0, -51.0, -133.0, -154.0, -147.0, -21.0, ...","[262.

In [119]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [120]:
raise ValueError()

ValueError: 